In [1]:
import six
import pandas as pd
import numpy as np
import math
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Bidirectional,TimeDistributed
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
def DoD(SPM):
    SoC_ = []
    SoC = 0
    for ct in range(0,len(SPM)):
        SoC = SoC + SPM[ct]
        SoC_.append(SoC)
    SoC_ = np.array(SoC_)
    return SoC_

In [3]:
#Load samsung cell data
# # 1/30C
# DFN_1_30C = np.array(pd.read_csv('Exp_data/Real_OCV_sam.csv',header = None))
# DFN_1_30C = DFN_1_30C.reshape(np.size(DFN_1_30C),1)
# NDC_1_30C = np.array(pd.read_csv('Exp_data/NDC_OCV_sam.csv',header = None))
# NDC_1_30C = np.concatenate((NDC_1_30C,np.ones((len(NDC_1_30C),1))*NDC_1_30C[0,1]),axis=1)

# 1C
DFN_1C = np.array(pd.read_csv('Journal_data/Exp/Real_1C_sam.csv',header = None))
# DFN_1C = DFN_1C.reshape(np.size(DFN_1C),1)
NDC_1C = np.array(pd.read_csv('Journal_data/Exp/NDC_1C_sam.csv',header = None))
# NDC_1C = np.concatenate((NDC_1C,np.arange(len(NDC_1C)).reshape(len(NDC_1C),1)),axis=1)

DoD_1C = DoD(-NDC_1C[0:len(NDC_1C),6]) # 4: SoC

NDC_1C_T = np.zeros((len(NDC_1C)-2,7))
NDC_1C_T[:,0] = NDC_1C[2:len(NDC_1C),7] # 7: Temperature (k)
NDC_1C_T[:,1] = -NDC_1C[2:len(NDC_1C),6] # 6: Current (k)
NDC_1C_T[:,2] = DoD_1C[2:len(NDC_1C)] # 4: SoC (k)
NDC_1C_T[:,3] = NDC_1C[1:len(NDC_1C)-1,7] # 7: Temperature (k-1)
NDC_1C_T[:,4] = -NDC_1C[1:len(NDC_1C)-1,6] # 4: Current (k-1)
NDC_1C_T[:,5] = DoD_1C[1:len(NDC_1C)-1] # 4: SoC (k-1)
NDC_1C_T[:,6] = DFN_1C[1:len(NDC_1C)-1,0] # 4: Voltage (k-1)
# NDC_1C_T[:,7] = DFN_1C[0:len(NDC_1C)-2,0] # 4: Voltage (k-2)

DFN_1C_T = DFN_1C[2:len(DFN_1C)] # Voltage (k)

# 2C
DFN_2C = np.array(pd.read_csv('Journal_data/Exp/Real_2C_sam.csv',header = None))
# DFN_2C = DFN_2C.reshape(np.size(DFN_2C),1)
NDC_2C = np.array(pd.read_csv('Journal_data/Exp/NDC_2C_sam.csv',header = None))
# NDC_2C = np.concatenate((NDC_2C,np.arange(len(NDC_2C)).reshape(len(NDC_2C),1)),axis=1)

DoD_2C = DoD(-NDC_2C[0:len(NDC_2C),6]) # 4: SoC

NDC_2C_T = np.zeros((len(NDC_2C)-2,7))
NDC_2C_T[:,0] = NDC_2C[2:len(NDC_2C),7] # 7: Temperature (k)
NDC_2C_T[:,1] = -NDC_2C[2:len(NDC_2C),6] # 6: Current (k)
NDC_2C_T[:,2] = DoD_2C[2:len(NDC_2C)] # 4: SoC (k)
NDC_2C_T[:,3] = NDC_2C[1:len(NDC_2C)-1,7] # 7: Temperature (k-1)
NDC_2C_T[:,4] = -NDC_2C[1:len(NDC_2C)-1,6] # 4: Current (k-1)
NDC_2C_T[:,5] = DoD_2C[1:len(NDC_2C)-1] # 4: SoC (k-1)
NDC_2C_T[:,6] = DFN_2C[1:len(NDC_2C)-1,0] # 4: Voltage (k-1)
# NDC_2C_T[:,7] = DFN_2C[0:len(NDC_2C)-2,0] # 4: Voltage (k-2)

DFN_2C_T = DFN_2C[2:len(DFN_2C)] # Voltage (k)


# 3C
DFN_3C = np.array(pd.read_csv('Journal_data/Exp/Real_3C_sam.csv',header = None))
# DFN_3C = DFN_3C.reshape(np.size(DFN_3C),1)
NDC_3C = np.array(pd.read_csv('Journal_data/Exp/NDC_3C_sam.csv',header = None))
# NDC_3C = np.concatenate((NDC_3C,np.arange(len(NDC_3C)).reshape(len(NDC_3C),1)),axis=1)

DoD_3C = DoD(-NDC_3C[0:len(NDC_3C),6]) # 4: SoC

NDC_3C_T = np.zeros((len(NDC_3C)-2,7))
NDC_3C_T[:,0] = NDC_3C[2:len(NDC_3C),7] # 7: Temperature (k)
NDC_3C_T[:,1] = -NDC_3C[2:len(NDC_3C),6] # 6: Current (k)
NDC_3C_T[:,2] = DoD_3C[2:len(NDC_3C)] # 4: SoC (k)
NDC_3C_T[:,3] = NDC_3C[1:len(NDC_3C)-1,7] # 7: Temperature (k-1)
NDC_3C_T[:,4] = -NDC_3C[1:len(NDC_3C)-1,6] # 4: Current (k-1)
NDC_3C_T[:,5] = DoD_3C[1:len(NDC_3C)-1] # 4: SoC (k-1)
NDC_3C_T[:,6] = DFN_3C[1:len(NDC_3C)-1,0] # 4: Voltage (k-1)
# NDC_3C_T[:,7] = DFN_3C[0:len(NDC_3C)-2,0] # 4: Voltage (k-2)

DFN_3C_T = DFN_3C[2:len(DFN_3C)] # Voltage (k)


# 4C
DFN_4C = np.array(pd.read_csv('Journal_data/Exp/Real_4C_sam.csv',header = None))
# DFN_4C = DFN_4C.reshape(np.size(DFN_4C),1)
NDC_4C = np.array(pd.read_csv('Journal_data/Exp/NDC_4C_sam.csv',header = None))
# NDC_4C = np.concatenate((NDC_4C,np.arange(len(NDC_4C)).reshape(len(NDC_4C),1)),axis=1)

DoD_4C = DoD(-NDC_4C[0:len(NDC_4C),6]) # 4: SoC

NDC_4C_T = np.zeros((len(NDC_4C)-2,7))
NDC_4C_T[:,0] = NDC_4C[2:len(NDC_4C),7] # 7: Temperature (k)
NDC_4C_T[:,1] = -NDC_4C[2:len(NDC_4C),6] # 6: Current (k)
NDC_4C_T[:,2] = DoD_4C[2:len(NDC_4C)] # 4: SoC (k)
NDC_4C_T[:,3] = NDC_4C[1:len(NDC_4C)-1,7] # 7: Temperature (k-1)
NDC_4C_T[:,4] = -NDC_4C[1:len(NDC_4C)-1,6] # 4: Current (k-1)
NDC_4C_T[:,5] = DoD_4C[1:len(NDC_4C)-1] # 4: SoC (k-1)
NDC_4C_T[:,6] = DFN_4C[1:len(NDC_4C)-1,0] # 4: Voltage (k-1)
# NDC_4C_T[:,7] = DFN_4C[0:len(NDC_4C)-2,0] # 4: Voltage (k-2)

DFN_4C_T = DFN_4C[2:len(DFN_4C)] # Voltage (k)

# 5C
DFN_5C = np.array(pd.read_csv('Journal_data/Exp/Real_5C_sam.csv',header = None))
# DFN_5C = DFN_5C.reshape(np.size(DFN_5C),1)
NDC_5C = np.array(pd.read_csv('Journal_data/Exp/NDC_5C_sam.csv',header = None))
# NDC_5C = np.concatenate((NDC_5C,np.arange(len(NDC_5C)).reshape(len(NDC_5C),1)),axis=1)

DoD_5C = DoD(-NDC_5C[0:len(NDC_5C),6]) # 4: SoC

NDC_5C_T = np.zeros((len(NDC_5C)-2,7))
NDC_5C_T[:,0] = NDC_5C[2:len(NDC_5C),7] # 7: Temperature (k)
NDC_5C_T[:,1] = -NDC_5C[2:len(NDC_5C),6] # 6: Current (k)
NDC_5C_T[:,2] = DoD_5C[2:len(NDC_5C)] # 4: SoC (k)
NDC_5C_T[:,3] = NDC_5C[1:len(NDC_5C)-1,7] # 7: Temperature (k-1)
NDC_5C_T[:,4] = -NDC_5C[1:len(NDC_5C)-1,6] # 4: Current (k-1)
NDC_5C_T[:,5] = DoD_5C[1:len(NDC_5C)-1] # 4: SoC (k-1)
NDC_5C_T[:,6] = DFN_5C[1:len(NDC_5C)-1,0] # 4: Voltage (k-1)
# NDC_5C_T[:,7] = DFN_5C[0:len(NDC_5C)-2,0] # 4: Voltage (k-2)

DFN_5C_T = DFN_5C[2:len(DFN_5C)] # Voltage (k)

# 6C
DFN_6C = np.array(pd.read_csv('Journal_data/Exp/Real_6C_sam.csv',header = None))
# DFN_6C = DFN_6C.reshape(np.size(DFN_6C),1)
NDC_6C = np.array(pd.read_csv('Journal_data/Exp/NDC_6C_sam.csv',header = None))
# NDC_6C = np.concatenate((NDC_6C,np.arange(len(NDC_6C)).reshape(len(NDC_6C),1)),axis=1)

DoD_6C = DoD(-NDC_6C[0:len(NDC_6C),6]) # 4: SoC

NDC_6C_T = np.zeros((len(NDC_6C)-2,7))
NDC_6C_T[:,0] = NDC_6C[2:len(NDC_6C),7] # 7: Temperature (k)
NDC_6C_T[:,1] = -NDC_6C[2:len(NDC_6C),6] # 6: Current (k)
NDC_6C_T[:,2] = DoD_6C[2:len(NDC_6C)] # 4: SoC (k)
NDC_6C_T[:,3] = NDC_6C[1:len(NDC_6C)-1,7] # 7: Temperature (k-1)
NDC_6C_T[:,4] = -NDC_6C[1:len(NDC_6C)-1,6] # 4: Current (k-1)
NDC_6C_T[:,5] = DoD_6C[1:len(NDC_6C)-1] # 4: SoC (k-1)
NDC_6C_T[:,6] = DFN_6C[1:len(NDC_6C)-1,0] # 4: Voltage (k-1)
# NDC_6C_T[:,7] = DFN_6C[0:len(NDC_6C)-2,0] # 4: Voltage (k-2)

DFN_6C_T = DFN_6C[2:len(DFN_6C)] # Voltage (k)

# 7C
DFN_7C = np.array(pd.read_csv('Journal_data/Exp/Real_7C_sam.csv',header = None))
# DFN_7C = DFN_7C.reshape(np.size(DFN_7C),1)
NDC_7C = np.array(pd.read_csv('Journal_data/Exp/NDC_7C_sam.csv',header = None))
# NDC_7C = np.concatenate((NDC_7C,np.arange(len(NDC_7C)).reshape(len(NDC_7C),1)),axis=1)

DoD_7C = DoD(-NDC_7C[0:len(NDC_7C),6]) # 4: SoC

NDC_7C_T = np.zeros((len(NDC_7C)-2,7))
NDC_7C_T[:,0] = NDC_7C[2:len(NDC_7C),7] # 7: Temperature (k)
NDC_7C_T[:,1] = -NDC_7C[2:len(NDC_7C),6] # 6: Current (k)
NDC_7C_T[:,2] = DoD_7C[2:len(NDC_7C)] # 4: SoC (k)
NDC_7C_T[:,3] = NDC_7C[1:len(NDC_7C)-1,7] # 7: Temperature (k-1)
NDC_7C_T[:,4] = -NDC_7C[1:len(NDC_7C)-1,6] # 4: Current (k-1)
NDC_7C_T[:,5] = DoD_7C[1:len(NDC_7C)-1] # 4: SoC (k-1)
NDC_7C_T[:,6] = DFN_7C[1:len(NDC_7C)-1,0] # 4: Voltage (k-1)
# NDC_7C_T[:,7] = DFN_7C[0:len(NDC_7C)-2,0] # 4: Voltage (k-2)

DFN_7C_T = DFN_7C[2:len(DFN_7C)] # Voltage (k)

# 8C
DFN_8C = np.array(pd.read_csv('Journal_data/Exp/Real_8C_sam.csv',header = None))
# DFN_8C = DFN_8C.reshape(np.size(DFN_8C),1)
NDC_8C = np.array(pd.read_csv('Journal_data/Exp/NDC_8C_sam.csv',header = None))
# NDC_8C = np.concatenate((NDC_8C,np.arange(len(NDC_8C)).reshape(len(NDC_8C),1)),axis=1)

DoD_8C = DoD(-NDC_8C[0:len(NDC_8C),6]) # 4: SoC

NDC_8C_T = np.zeros((len(NDC_8C)-2,7))
NDC_8C_T[:,0] = NDC_8C[2:len(NDC_8C),7] # 7: Temperature (k)
NDC_8C_T[:,1] = -NDC_8C[2:len(NDC_8C),6] # 6: Current (k)
NDC_8C_T[:,2] = DoD_8C[2:len(NDC_8C)] # 4: SoC (k)
NDC_8C_T[:,3] = NDC_8C[1:len(NDC_8C)-1,7] # 7: Temperature (k-1)
NDC_8C_T[:,4] = -NDC_8C[1:len(NDC_8C)-1,6] # 4: Current (k-1)
NDC_8C_T[:,5] = DoD_8C[1:len(NDC_8C)-1] # 4: SoC (k-1)
NDC_8C_T[:,6] = DFN_8C[1:len(NDC_8C)-1,0] # 4: Voltage (k-1)
# NDC_8C_T[:,7] = DFN_8C[0:len(NDC_8C)-2,0] # 4: Voltage (k-2)

DFN_8C_T = DFN_8C[2:len(DFN_8C)] # Voltage (k)

# UDDS1
DFN_UDDS1 = np.array(pd.read_csv('Journal_data/Exp/Real_UDDS2_sam.csv',header = None))
# DFN_UDDS1 = DFN_UDDS1.reshape(np.size(DFN_UDDS1),1)
NDC_UDDS1 = np.array(pd.read_csv('Journal_data/Exp/NDC_UDDS2_sam.csv',header = None))
# NDC_UDDS1 = np.concatenate((NDC_UDDS1,np.arange(len(NDC_UDDS1)).reshape(len(NDC_UDDS1),1)),axis=1)

DoD_UDDS1 = DoD(-NDC_UDDS1[0:len(NDC_UDDS1),6]) # 4: SoC

NDC_UDDS1_T = np.zeros((len(NDC_UDDS1)-2,7))
NDC_UDDS1_T[:,0] = NDC_UDDS1[2:len(NDC_UDDS1),7] # 7: Temperature (k)
NDC_UDDS1_T[:,1] = -NDC_UDDS1[2:len(NDC_UDDS1),6] # 6: Current (k)
NDC_UDDS1_T[:,2] = DoD_UDDS1[2:len(NDC_UDDS1)] # 4: SoC (k)
NDC_UDDS1_T[:,3] = NDC_UDDS1[1:len(NDC_UDDS1)-1,7] # 7: Temperature (k-1)
NDC_UDDS1_T[:,4] = -NDC_UDDS1[1:len(NDC_UDDS1)-1,6] # 4: Current (k-1)
NDC_UDDS1_T[:,5] = DoD_UDDS1[1:len(NDC_UDDS1)-1] # 4: SoC (k-1)
NDC_UDDS1_T[:,6] = DFN_UDDS1[1:len(NDC_UDDS1)-1,0] # 4: Voltage (k-1)
# NDC_UDDS1_T[:,7] = DFN_UDDS1[0:len(NDC_UDDS1)-2,0] # 4: Voltage (k-2)

DFN_UDDS1_T = DFN_UDDS1[2:len(DFN_UDDS1)] # Voltage (k)

# US06_1
DFN_US061 = np.array(pd.read_csv('Journal_data/Exp/Real_US062_sam.csv',header = None))
# DFN_US061 = DFN_US061.reshape(np.size(DFN_US061),1)
NDC_US061 = np.array(pd.read_csv('Journal_data/Exp/NDC_US062_sam.csv',header = None))
# NDC_US061 = np.concatenate((NDC_US061,np.arange(len(NDC_US061)).reshape(len(NDC_US061),1)),axis=1)

DoD_US061 = DoD(-NDC_US061[0:len(NDC_US061),6]) # 4: SoC

NDC_US061_T = np.zeros((len(NDC_US061)-2,7))
NDC_US061_T[:,0] = NDC_US061[2:len(NDC_US061),7] # 7: Temperature (k)
NDC_US061_T[:,1] = -NDC_US061[2:len(NDC_US061),6] # 6: Current (k)
NDC_US061_T[:,2] = DoD_US061[2:len(NDC_US061)] # 4: SoC (k)
NDC_US061_T[:,3] = NDC_US061[1:len(NDC_US061)-1,7] # 7: Temperature (k-1)
NDC_US061_T[:,4] = -NDC_US061[1:len(NDC_US061)-1,6] # 4: Current (k-1)
NDC_US061_T[:,5] = DoD_US061[1:len(NDC_US061)-1] # 4: SoC (k-1)
NDC_US061_T[:,6] = DFN_US061[1:len(NDC_US061)-1,0] # 4: Voltage (k-1)
# NDC_US061_T[:,7] = DFN_US061[0:len(NDC_US061)-2,0] # 4: Voltage (k-2)

DFN_US061_T = DFN_US061[2:len(DFN_US061)] # Voltage (k)

# LA92_1
DFN_LA921 = np.array(pd.read_csv('Journal_data/Exp/Real_LA922_sam.csv',header = None))
# DFN_LA921 = DFN_LA921.reshape(np.size(DFN_LA921),1)
NDC_LA921 = np.array(pd.read_csv('Journal_data/Exp/NDC_LA922_sam.csv',header = None))
# NDC_LA921 = np.concatenate((NDC_LA921,np.arange(len(NDC_LA921)).reshape(len(NDC_LA921),1)),axis=1)

DoD_LA921 = DoD(-NDC_LA921[0:len(NDC_LA921),6]) # 4: SoC

NDC_LA921_T = np.zeros((len(NDC_LA921)-2,7))
NDC_LA921_T[:,0] = NDC_LA921[2:len(NDC_LA921),7] # 7: Temperature (k)
NDC_LA921_T[:,1] = -NDC_LA921[2:len(NDC_LA921),6] # 6: Current (k)
NDC_LA921_T[:,2] = DoD_LA921[2:len(NDC_LA921)] # 4: SoC (k)
NDC_LA921_T[:,3] = NDC_LA921[1:len(NDC_LA921)-1,7] # 7: Temperature (k-1)
NDC_LA921_T[:,4] = -NDC_LA921[1:len(NDC_LA921)-1,6] # 4: Current (k-1)
NDC_LA921_T[:,5] = DoD_LA921[1:len(NDC_LA921)-1] # 4: SoC (k-1)
NDC_LA921_T[:,6] = DFN_LA921[1:len(NDC_LA921)-1,0] # 4: Voltage (k-1)
# NDC_LA921_T[:,7] = DFN_LA921[0:len(NDC_LA921)-2,0] # 4: Voltage (k-2)

DFN_LA921_T = DFN_LA921[2:len(DFN_LA921)] # Voltage (k)

# SC04_1
DFN_SC041 = np.array(pd.read_csv('Journal_data/Exp/Real_SC042_sam.csv',header = None))
# DFN_SC041 = DFN_SC041.reshape(np.size(DFN_SC041),1)
NDC_SC041 = np.array(pd.read_csv('Journal_data/Exp/NDC_SC042_sam.csv',header = None))
# NDC_SC041 = np.concatenate((NDC_SC041,np.arange(len(NDC_SC041)).reshape(len(NDC_SC041),1)),axis=1)

DoD_SC041 = DoD(-NDC_SC041[0:len(NDC_SC041),6]) # 4: SoC

NDC_SC041_T = np.zeros((len(NDC_SC041)-2,7))
NDC_SC041_T[:,0] = NDC_SC041[2:len(NDC_SC041),7] # 7: Temperature (k)
NDC_SC041_T[:,1] = -NDC_SC041[2:len(NDC_SC041),6] # 6: Current (k)
NDC_SC041_T[:,2] = DoD_SC041[2:len(NDC_SC041)] # 4: SoC (k)
NDC_SC041_T[:,3] = NDC_SC041[1:len(NDC_SC041)-1,7] # 7: Temperature (k-1)
NDC_SC041_T[:,4] = -NDC_SC041[1:len(NDC_SC041)-1,6] # 4: Current (k-1)
NDC_SC041_T[:,5] = DoD_SC041[1:len(NDC_SC041)-1] # 4: SoC (k-1)
NDC_SC041_T[:,6] = DFN_SC041[1:len(NDC_SC041)-1,0] # 4: Voltage (k-1)
# NDC_SC041_T[:,7] = DFN_SC041[0:len(NDC_SC041)-2,0] # 4: Voltage (k-2)

DFN_SC041_T = DFN_SC041[2:len(DFN_SC041)] # Voltage (k)


# 1C
DFN_1C_off = np.array(pd.read_csv('Journal_data/Exp/Real_1C_sam_off.csv',header = None))
# DFN_1C_off = DFN_1C_off.reshape(np.size(DFN_1C_off),1)
NDC_1C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_1C_sam_off.csv',header = None))
# NDC_1C_off = np.concatenate((NDC_1C_off,np.arange(len(NDC_1C_off)).reshape(len(NDC_1C_off),1)),axis=1)

DoD_1C_off = DoD(-NDC_1C_off[0:len(NDC_1C_off),6]) # 4: SoC

NDC_1C_off_T = np.zeros((len(NDC_1C_off)-2,7))
NDC_1C_off_T[:,0] = NDC_1C_off[2:len(NDC_1C_off),7] # 7: Temperature (k)
NDC_1C_off_T[:,1] = -NDC_1C_off[2:len(NDC_1C_off),6] # 6: Current (k)
NDC_1C_off_T[:,2] = DoD_1C_off[2:len(NDC_1C_off)] # 4: SoC (k)
NDC_1C_off_T[:,3] = NDC_1C_off[1:len(NDC_1C_off)-1,7] # 7: Temperature (k-1)
NDC_1C_off_T[:,4] = -NDC_1C_off[1:len(NDC_1C_off)-1,6] # 4: Current (k-1)
NDC_1C_off_T[:,5] = DoD_1C_off[1:len(NDC_1C_off)-1] # 4: SoC (k-1)
NDC_1C_off_T[:,6] = DFN_1C_off[1:len(NDC_1C_off)-1,0] # 4: Voltage (k-1)
# NDC_1C_off_T[:,7] = DFN_1C_off[0:len(NDC_1C_off)-2,0] # 4: Voltage (k-2)

DFN_1C_off_T = DFN_1C_off[2:len(DFN_1C_off)] # Voltage (k)


# 2C
DFN_2C_off = np.array(pd.read_csv('Journal_data/Exp/Real_2C_sam_off.csv',header = None))
# DFN_2C_off = DFN_2C_off.reshape(np.size(DFN_2C_off),1)
NDC_2C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_2C_sam_off.csv',header = None))
# NDC_2C_off = np.concatenate((NDC_2C_off,np.arange(len(NDC_2C_off)).reshape(len(NDC_2C_off),1)),axis=1)

DoD_2C_off = DoD(-NDC_2C_off[0:len(NDC_2C_off),6]) # 4: SoC

NDC_2C_off_T = np.zeros((len(NDC_2C_off)-2,7))
NDC_2C_off_T[:,0] = NDC_2C_off[2:len(NDC_2C_off),7] # 7: Temperature (k)
NDC_2C_off_T[:,1] = -NDC_2C_off[2:len(NDC_2C_off),6] # 6: Current (k)
NDC_2C_off_T[:,2] = DoD_2C_off[2:len(NDC_2C_off)] # 4: SoC (k)
NDC_2C_off_T[:,3] = NDC_2C_off[1:len(NDC_2C_off)-1,7] # 7: Temperature (k-1)
NDC_2C_off_T[:,4] = -NDC_2C_off[1:len(NDC_2C_off)-1,6] # 4: Current (k-1)
NDC_2C_off_T[:,5] = DoD_2C_off[1:len(NDC_2C_off)-1] # 4: SoC (k-1)
NDC_2C_off_T[:,6] = DFN_2C_off[1:len(NDC_2C_off)-1,0] # 4: Voltage (k-1)
# NDC_2C_off_T[:,7] = DFN_2C_off[0:len(NDC_2C_off)-2,0] # 4: Voltage (k-2)

DFN_2C_off_T = DFN_2C_off[2:len(DFN_2C_off)] # Voltage (k)


# 3C
DFN_3C_off = np.array(pd.read_csv('Journal_data/Exp/Real_3C_sam_off.csv',header = None))
# DFN_3C_off = DFN_3C_off.reshape(np.size(DFN_3C_off),1)
NDC_3C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_3C_sam_off.csv',header = None))
# NDC_3C_off = np.concatenate((NDC_3C_off,np.arange(len(NDC_3C_off)).reshape(len(NDC_3C_off),1)),axis=1)

DoD_3C_off = DoD(-NDC_3C_off[0:len(NDC_3C_off),6]) # 4: SoC

NDC_3C_off_T = np.zeros((len(NDC_3C_off)-2,7))
NDC_3C_off_T[:,0] = NDC_3C_off[2:len(NDC_3C_off),7] # 7: Temperature (k)
NDC_3C_off_T[:,1] = -NDC_3C_off[2:len(NDC_3C_off),6] # 6: Current (k)
NDC_3C_off_T[:,2] = DoD_3C_off[2:len(NDC_3C_off)] # 4: SoC (k)
NDC_3C_off_T[:,3] = NDC_3C_off[1:len(NDC_3C_off)-1,7] # 7: Temperature (k-1)
NDC_3C_off_T[:,4] = -NDC_3C_off[1:len(NDC_3C_off)-1,6] # 4: Current (k-1)
NDC_3C_off_T[:,5] = DoD_3C_off[1:len(NDC_3C_off)-1] # 4: SoC (k-1)
NDC_3C_off_T[:,6] = DFN_3C_off[1:len(NDC_3C_off)-1,0] # 4: Voltage (k-1)
# NDC_3C_off_T[:,7] = DFN_3C_off[0:len(NDC_3C_off)-2,0] # 4: Voltage (k-2)

DFN_3C_off_T = DFN_3C_off[2:len(DFN_3C_off)] # Voltage (k)


# 4C
DFN_4C_off = np.array(pd.read_csv('Journal_data/Exp/Real_4C_sam_off.csv',header = None))
# DFN_4C_off = DFN_4C_off.reshape(np.size(DFN_4C_off),1)
NDC_4C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_4C_sam_off.csv',header = None))
# NDC_4C_off = np.concatenate((NDC_4C_off,np.arange(len(NDC_4C_off)).reshape(len(NDC_4C_off),1)),axis=1)

DoD_4C_off = DoD(-NDC_4C_off[0:len(NDC_4C_off),6]) # 4: SoC

NDC_4C_off_T = np.zeros((len(NDC_4C_off)-2,7))
NDC_4C_off_T[:,0] = NDC_4C_off[2:len(NDC_4C_off),7] # 7: Temperature (k)
NDC_4C_off_T[:,1] = -NDC_4C_off[2:len(NDC_4C_off),6] # 6: Current (k)
NDC_4C_off_T[:,2] = DoD_4C_off[2:len(NDC_4C_off)] # 4: SoC (k)
NDC_4C_off_T[:,3] = NDC_4C_off[1:len(NDC_4C_off)-1,7] # 7: Temperature (k-1)
NDC_4C_off_T[:,4] = -NDC_4C_off[1:len(NDC_4C_off)-1,6] # 4: Current (k-1)
NDC_4C_off_T[:,5] = DoD_4C_off[1:len(NDC_4C_off)-1] # 4: SoC (k-1)
NDC_4C_off_T[:,6] = DFN_4C_off[1:len(NDC_4C_off)-1,0] # 4: Voltage (k-1)
# NDC_4C_off_T[:,7] = DFN_4C_off[0:len(NDC_4C_off)-2,0] # 4: Voltage (k-2)

DFN_4C_off_T = DFN_4C_off[2:len(DFN_4C_off)] # Voltage (k)


# 5C
DFN_5C_off = np.array(pd.read_csv('Journal_data/Exp/Real_5C_sam_off.csv',header = None))
# DFN_5C_off = DFN_5C_off.reshape(np.size(DFN_5C_off),1)
NDC_5C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_5C_sam_off.csv',header = None))
# NDC_5C_off = np.concatenate((NDC_5C_off,np.arange(len(NDC_5C_off)).reshape(len(NDC_5C_off),1)),axis=1)

DoD_5C_off = DoD(-NDC_5C_off[0:len(NDC_5C_off),6]) # 4: SoC

NDC_5C_off_T = np.zeros((len(NDC_5C_off)-2,7))
NDC_5C_off_T[:,0] = NDC_5C_off[2:len(NDC_5C_off),7] # 7: Temperature (k)
NDC_5C_off_T[:,1] = -NDC_5C_off[2:len(NDC_5C_off),6] # 6: Current (k)
NDC_5C_off_T[:,2] = DoD_5C_off[2:len(NDC_5C_off)] # 4: SoC (k)
NDC_5C_off_T[:,3] = NDC_5C_off[1:len(NDC_5C_off)-1,7] # 7: Temperature (k-1)
NDC_5C_off_T[:,4] = -NDC_5C_off[1:len(NDC_5C_off)-1,6] # 4: Current (k-1)
NDC_5C_off_T[:,5] = DoD_5C_off[1:len(NDC_5C_off)-1] # 4: SoC (k-1)
NDC_5C_off_T[:,6] = DFN_5C_off[1:len(NDC_5C_off)-1,0] # 4: Voltage (k-1)
# NDC_5C_off_T[:,7] = DFN_5C_off[0:len(NDC_5C_off)-2,0] # 4: Voltage (k-2)

DFN_5C_off_T = DFN_5C_off[2:len(DFN_5C_off)] # Voltage (k)


# 6C
DFN_6C_off = np.array(pd.read_csv('Journal_data/Exp/Real_6C_sam_off.csv',header = None))
# DFN_6C_off = DFN_6C_off.reshape(np.size(DFN_6C_off),1)
NDC_6C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_6C_sam_off.csv',header = None))
# NDC_6C_off = np.concatenate((NDC_6C_off,np.arange(len(NDC_6C_off)).reshape(len(NDC_6C_off),1)),axis=1)

DoD_6C_off = DoD(-NDC_6C_off[0:len(NDC_6C_off),6]) # 4: SoC

NDC_6C_off_T = np.zeros((len(NDC_6C_off)-2,7))
NDC_6C_off_T[:,0] = NDC_6C_off[2:len(NDC_6C_off),7] # 7: Temperature (k)
NDC_6C_off_T[:,1] = -NDC_6C_off[2:len(NDC_6C_off),6] # 6: Current (k)
NDC_6C_off_T[:,2] = DoD_6C_off[2:len(NDC_6C_off)] # 4: SoC (k)
NDC_6C_off_T[:,3] = NDC_6C_off[1:len(NDC_6C_off)-1,7] # 7: Temperature (k-1)
NDC_6C_off_T[:,4] = -NDC_6C_off[1:len(NDC_6C_off)-1,6] # 4: Current (k-1)
NDC_6C_off_T[:,5] = DoD_6C_off[1:len(NDC_6C_off)-1] # 4: SoC (k-1)
NDC_6C_off_T[:,6] = DFN_6C_off[1:len(NDC_6C_off)-1,0] # 4: Voltage (k-1)
# NDC_6C_off_T[:,7] = DFN_6C_off[0:len(NDC_6C_off)-2,0] # 4: Voltage (k-1)

DFN_6C_off_T = DFN_6C_off[2:len(DFN_6C_off)] # Voltage (k)

# 7C
DFN_7C_off = np.array(pd.read_csv('Journal_data/Exp/Real_7C_sam_off.csv',header = None))
# DFN_7C_off = DFN_7C_off.reshape(np.size(DFN_7C_off),1)
NDC_7C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_7C_sam_off.csv',header = None))
# NDC_7C_off = np.concatenate((NDC_7C_off,np.arange(len(NDC_7C_off)).reshape(len(NDC_7C_off),1)),axis=1)

DoD_7C_off = DoD(-NDC_7C_off[0:len(NDC_7C_off),6]) # 4: SoC

NDC_7C_off_T = np.zeros((len(NDC_7C_off)-2,7))
NDC_7C_off_T[:,0] = NDC_7C_off[2:len(NDC_7C_off),7] # 7: Temperature (k)
NDC_7C_off_T[:,1] = -NDC_7C_off[2:len(NDC_7C_off),6] # 6: Current (k)
NDC_7C_off_T[:,2] = DoD_7C_off[2:len(NDC_7C_off)] # 4: SoC (k)
NDC_7C_off_T[:,3] = NDC_7C_off[1:len(NDC_7C_off)-1,7] # 7: Temperature (k-1)
NDC_7C_off_T[:,4] = -NDC_7C_off[1:len(NDC_7C_off)-1,6] # 4: Current (k-1)
NDC_7C_off_T[:,5] = DoD_7C_off[1:len(NDC_7C_off)-1] # 4: SoC (k-1)
NDC_7C_off_T[:,6] = DFN_7C_off[1:len(NDC_7C_off)-1,0] # 4: Voltage (k-1)
# NDC_7C_off_T[:,7] = DFN_7C_off[0:len(NDC_7C_off)-2,0] # 4: Voltage (k-1)

DFN_7C_off_T = DFN_7C_off[2:len(DFN_7C_off)] # Voltage (k)

# 8C
DFN_8C_off = np.array(pd.read_csv('Journal_data/Exp/Real_8C_sam_off.csv',header = None))
# DFN_8C_off = DFN_8C_off.reshape(np.size(DFN_8C_off),1)
NDC_8C_off = np.array(pd.read_csv('Journal_data/Exp/NDC_8C_sam_off.csv',header = None))
# NDC_8C_off = np.concatenate((NDC_8C_off,np.arange(len(NDC_8C_off)).reshape(len(NDC_8C_off),1)),axis=1)

DoD_8C_off = DoD(-NDC_8C_off[0:len(NDC_8C_off),6]) # 4: SoC

NDC_8C_off_T = np.zeros((len(NDC_8C_off)-2,7))
NDC_8C_off_T[:,0] = NDC_8C_off[2:len(NDC_8C_off),7] # 7: Temperature (k)
NDC_8C_off_T[:,1] = -NDC_8C_off[2:len(NDC_8C_off),6] # 6: Current (k)
NDC_8C_off_T[:,2] = DoD_8C_off[2:len(NDC_8C_off)] # 4: SoC (k)
NDC_8C_off_T[:,3] = NDC_8C_off[1:len(NDC_8C_off)-1,7] # 7: Temperature (k-1)
NDC_8C_off_T[:,4] = -NDC_8C_off[1:len(NDC_8C_off)-1,6] # 4: Current (k-1)
NDC_8C_off_T[:,5] = DoD_8C_off[1:len(NDC_8C_off)-1] # 4: SoC (k-1)
NDC_8C_off_T[:,6] = DFN_8C_off[1:len(NDC_8C_off)-1,0] # 4: Voltage (k-1)
# NDC_8C_off_T[:,7] = DFN_8C_off[0:len(NDC_8C_off)-2,0] # 4: Voltage (k-1)

DFN_8C_off_T = DFN_8C_off[2:len(DFN_8C_off)] # Voltage (k)


# UDDS1
DFN_UDDS2_off = np.array(pd.read_csv('Journal_data/Exp/Real_UDDS2_sam_off.csv',header = None))
# DFN_UDDS2_off = DFN_UDDS2_off.reshape(np.size(DFN_UDDS2_off),1)
NDC_UDDS2_off = np.array(pd.read_csv('Journal_data/Exp/NDC_UDDS2_sam_off.csv',header = None))
# NDC_UDDS2_off = np.concatenate((NDC_UDDS2_off,np.arange(len(NDC_UDDS2_off)).reshape(len(NDC_UDDS2_off),1)),axis=1)

DoD_UDDS2_off = DoD(-NDC_UDDS2_off[0:len(NDC_UDDS2_off),6]) # 4: SoC

NDC_UDDS2_off_T = np.zeros((len(NDC_UDDS2_off)-2,7))
NDC_UDDS2_off_T[:,0] = NDC_UDDS2_off[2:len(NDC_UDDS2_off),7] # 7: Temperature (k)
NDC_UDDS2_off_T[:,1] = -NDC_UDDS2_off[2:len(NDC_UDDS2_off),6] # 6: Current (k)
NDC_UDDS2_off_T[:,2] = DoD_UDDS2_off[2:len(NDC_UDDS2_off)] # 4: SoC (k)
NDC_UDDS2_off_T[:,3] = NDC_UDDS2_off[1:len(NDC_UDDS2_off)-1,7] # 7: Temperature (k-1)
NDC_UDDS2_off_T[:,4] = -NDC_UDDS2_off[1:len(NDC_UDDS2_off)-1,6] # 4: Current (k-1)
NDC_UDDS2_off_T[:,5] = DoD_UDDS2_off[1:len(NDC_UDDS2_off)-1] # 4: SoC (k-1)
NDC_UDDS2_off_T[:,6] = DFN_UDDS2_off[1:len(NDC_UDDS2_off)-1,0] # 4: Voltage (k-1)
# NDC_UDDS2_off_T[:,7] = DFN_UDDS2_off[0:len(NDC_UDDS2_off)-2,0] # 4: Voltage (k-1)

DFN_UDDS2_off_T = DFN_UDDS2_off[2:len(DFN_UDDS2_off)] # Voltage (k)

# US06_1
DFN_US062_off = np.array(pd.read_csv('Journal_data/Exp/Real_US062_sam_off.csv',header = None))
# DFN_US062_off = DFN_US062_off.reshape(np.size(DFN_US062_off),1)
NDC_US062_off = np.array(pd.read_csv('Journal_data/Exp/NDC_US062_sam_off.csv',header = None))
# NDC_US062_off = np.concatenate((NDC_US062_off,np.arange(len(NDC_US062_off)).reshape(len(NDC_US062_off),1)),axis=1)

DoD_US062_off = DoD(-NDC_US062_off[0:len(NDC_US062_off),6]) # 4: SoC

NDC_US062_off_T = np.zeros((len(NDC_US062_off)-2,7))
NDC_US062_off_T[:,0] = NDC_US062_off[2:len(NDC_US062_off),7] # 7: Temperature (k)
NDC_US062_off_T[:,1] = -NDC_US062_off[2:len(NDC_US062_off),6] # 6: Current (k)
NDC_US062_off_T[:,2] = DoD_US062_off[2:len(NDC_US062_off)] # 4: SoC (k)
NDC_US062_off_T[:,3] = NDC_US062_off[1:len(NDC_US062_off)-1,7] # 7: Temperature (k-1)
NDC_US062_off_T[:,4] = -NDC_US062_off[1:len(NDC_US062_off)-1,6] # 4: Current (k-1)
NDC_US062_off_T[:,5] = DoD_US062_off[1:len(NDC_US062_off)-1] # 4: SoC (k-1)
NDC_US062_off_T[:,6] = DFN_US062_off[1:len(NDC_US062_off)-1,0] # 4: Voltage (k-1)
# NDC_US062_off_T[:,7] = DFN_US062_off[0:len(NDC_US062_off)-2,0] # 4: Voltage (k-1)

DFN_US062_off_T = DFN_US062_off[2:len(DFN_US062_off)] # Voltage (k)


# LA92_1
DFN_LA922_off = np.array(pd.read_csv('Journal_data/Exp/Real_LA922_sam_off.csv',header = None))
# DFN_LA922_off = DFN_LA922_off.reshape(np.size(DFN_LA922_off),1)
NDC_LA922_off = np.array(pd.read_csv('Journal_data/Exp/NDC_LA922_sam_off.csv',header = None))
# NDC_LA922_off = np.concatenate((NDC_LA922_off,np.arange(len(NDC_LA922_off)).reshape(len(NDC_LA922_off),1)),axis=1)

DoD_LA922_off = DoD(-NDC_LA922_off[0:len(NDC_LA922_off),6]) # 4: SoC

NDC_LA922_off_T = np.zeros((len(NDC_LA922_off)-2,7))
NDC_LA922_off_T[:,0] = NDC_LA922_off[2:len(NDC_LA922_off),7] # 7: Temperature (k)
NDC_LA922_off_T[:,1] = -NDC_LA922_off[2:len(NDC_LA922_off),6] # 6: Current (k)
NDC_LA922_off_T[:,2] = DoD_LA922_off[2:len(NDC_LA922_off)] # 4: SoC (k)
NDC_LA922_off_T[:,3] = NDC_LA922_off[1:len(NDC_LA922_off)-1,7] # 7: Temperature (k-1)
NDC_LA922_off_T[:,4] = -NDC_LA922_off[1:len(NDC_LA922_off)-1,6] # 4: Current (k-1)
NDC_LA922_off_T[:,5] = DoD_LA922_off[1:len(NDC_LA922_off)-1] # 4: SoC (k-1)
NDC_LA922_off_T[:,6] = DFN_LA922_off[1:len(NDC_LA922_off)-1,0] # 4: Voltage (k-1)
# NDC_LA922_off_T[:,7] = DFN_LA922_off[0:len(NDC_LA922_off)-2,0] # 4: Voltage (k-2)

DFN_LA922_off_T = DFN_LA922_off[2:len(DFN_LA922_off)] # Voltage (k)


# SC04_2
DFN_SC042_off = np.array(pd.read_csv('Journal_data/Exp/Real_SC042_sam_off.csv',header = None))
# DFN_SC042_off = DFN_SC042_off.reshape(np.size(DFN_SC042_off),1)
NDC_SC042_off = np.array(pd.read_csv('Journal_data/Exp/NDC_SC042_sam_off.csv',header = None))
# NDC_SC042_off = np.concatenate((NDC_SC042_off,np.arange(len(NDC_SC042_off)).reshape(len(NDC_SC042_off),1)),axis=1)

DoD_SC042_off = DoD(-NDC_SC042_off[0:len(NDC_SC042_off),6]) # 4: SoC

NDC_SC042_off_T = np.zeros((len(NDC_SC042_off)-2,7))
NDC_SC042_off_T[:,0] = NDC_SC042_off[2:len(NDC_SC042_off),7] # 7: Temperature (k)
NDC_SC042_off_T[:,1] = -NDC_SC042_off[2:len(NDC_SC042_off),6] # 6: Current (k)
NDC_SC042_off_T[:,2] = DoD_SC042_off[2:len(NDC_SC042_off)] # 4: SoC (k)
NDC_SC042_off_T[:,3] = NDC_SC042_off[1:len(NDC_SC042_off)-1,7] # 7: Temperature (k-1)
NDC_SC042_off_T[:,4] = -NDC_SC042_off[1:len(NDC_SC042_off)-1,6] # 4: Current (k-1)
NDC_SC042_off_T[:,5] = DoD_SC042_off[1:len(NDC_SC042_off)-1] # 4: SoC (k-1)
NDC_SC042_off_T[:,6] = DFN_SC042_off[1:len(NDC_SC042_off)-1,0] # 4: Voltage (k-1)
# NDC_SC042_off_T[:,7] = DFN_SC042_off[0:len(NDC_SC042_off)-2,0] # 4: Voltage (k-2)

DFN_SC042_off_T = DFN_SC042_off[2:len(DFN_SC042_off)] # Voltage (k)


DFN = np.concatenate((DFN_1C_T,DFN_2C_T),axis=0)
# DFN = np.concatenate((DFN,DFN_3C_T),axis=0)
# DFN = np.concatenate((DFN,DFN_4C_T),axis=0)
DFN = np.concatenate((DFN,DFN_5C_T),axis=0)
# DFN = np.concatenate((DFN,DFN_6C_T),axis=0)
DFN = np.concatenate((DFN,DFN_7C_T),axis=0)
DFN = np.concatenate((DFN,DFN_8C_T),axis=0)
# DFN = np.concatenate((DFN,DFN_UDDS1_T),axis=0)
DFN = np.concatenate((DFN,DFN_US061_T),axis=0)
# DFN = np.concatenate((DFN,DFN_LA921_T),axis=0)
DFN = np.concatenate((DFN,DFN_SC041_T),axis=0)

SPM = np.concatenate((NDC_1C_T,NDC_2C_T),axis=0)
# SPM = np.concatenate((SPM,NDC_3C_T),axis=0)
# SPM = np.concatenate((SPM,NDC_4C_T),axis=0)
SPM = np.concatenate((SPM,NDC_5C_T),axis=0)
# SPM = np.concatenate((SPM,NDC_6C_T),axis=0)
SPM = np.concatenate((SPM,NDC_7C_T),axis=0)
SPM = np.concatenate((SPM,NDC_8C_T),axis=0)
# SPM = np.concatenate((SPM,NDC_UDDS1_T),axis=0)
SPM = np.concatenate((SPM,NDC_US061_T),axis=0)
# SPM = np.concatenate((SPM,NDC_LA921_T),axis=0)
SPM = np.concatenate((SPM,NDC_SC041_T),axis=0)

DFN = np.concatenate((DFN,DFN_1C_off_T),axis=0)
DFN = np.concatenate((DFN,DFN_2C_off_T),axis=0)
# DFN = np.concatenate((DFN,DFN_3C_off_T),axis=0)
# DFN = np.concatenate((DFN,DFN_4C_off_T),axis=0)
DFN = np.concatenate((DFN,DFN_5C_off_T),axis=0)
# DFN = np.concatenate((DFN,DFN_6C_off_T),axis=0)
DFN = np.concatenate((DFN,DFN_7C_off_T),axis=0)
DFN = np.concatenate((DFN,DFN_8C_off_T),axis=0)
# DFN = np.concatenate((DFN,DFN_UDDS2_off_T),axis=0)
DFN = np.concatenate((DFN,DFN_US062_off_T),axis=0)
# DFN = np.concatenate((DFN,DFN_LA922_off_T),axis=0)
DFN = np.concatenate((DFN,DFN_SC042_off_T),axis=0)

SPM = np.concatenate((SPM,NDC_1C_off_T),axis=0)
SPM = np.concatenate((SPM,NDC_2C_off_T),axis=0)
# SPM = np.concatenate((SPM,NDC_3C_off_T),axis=0)
# SPM = np.concatenate((SPM,NDC_4C_off_T),axis=0)
SPM = np.concatenate((SPM,NDC_5C_off_T),axis=0)
# SPM = np.concatenate((SPM,NDC_6C_off_T),axis=0)
SPM = np.concatenate((SPM,NDC_7C_off_T),axis=0)
SPM = np.concatenate((SPM,NDC_8C_off_T),axis=0)
# SPM = np.concatenate((SPM,NDC_UDDS2_off_T),axis=0)
SPM = np.concatenate((SPM,NDC_US062_off_T),axis=0)
# SPM = np.concatenate((SPM,NDC_LA922_off_T),axis=0)
SPM = np.concatenate((SPM,NDC_SC042_off_T),axis=0)

In [4]:
print("Concatenate SPM train length: " + str(len(SPM)))
print("Concatenate DFN train length: " + str(len(DFN)))

Concatenate SPM train length: 31796
Concatenate DFN train length: 31796


In [5]:
#Construct training data
trainX = np.zeros((len(SPM),7))
trainX[:,0] = SPM[:,0] # 0: Vt
trainX[:,1] = SPM[:,1] # 1: Vb
trainX[:,2] = SPM[:,2] # 2: Vs
trainX[:,3] = SPM[:,3] # 3: V1
trainX[:,4] = SPM[:,4] # 4: SoC
trainX[:,5] = SPM[:,5] # 5: R0
trainX[:,6] = SPM[:,6] # 6: Current

# trainX = SPM
trainY = DFN

In [6]:
scaler1 = MinMaxScaler()
scaler1.fit(trainX)
normalizedX = scaler1.transform(trainX)
scaler2 = MinMaxScaler()
scaler2.fit(trainY)
normalizedY = scaler2.transform(trainY)
trainX = normalizedX
trainY = normalizedY

In [27]:
model = Sequential()

model.add(Dense(256, input_dim=7, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               2048      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 68,097
Trainable params: 68,097
Non-trainable params: 0
_________________________________________________________________


In [79]:
# train NN
model.fit(trainX, trainY, epochs=1, verbose=0)
model.evaluate(trainX, trainY)

31796/31796 [==============================] - 1s 32us/sample - loss: 1.9004e-06


1.9004097887829005e-06

In [80]:
trainPredict = model.predict(trainX)
trainPredict = scaler2.inverse_transform(trainPredict.reshape(len(trainPredict),1))
trainScore = math.sqrt(mean_squared_error(DFN, trainPredict.reshape(len(trainPredict),1)))
print("Train Score: RMSE " + str(trainScore))

Train Score: RMSE 0.0026886765083664627


In [75]:
# Result Analysis
#Construct training data
SPMTT = NDC_US062_off_T
# SPMTT = np.concatenate((SPMTT,NDC_LA922_off),axis=0)
DFNTT = DFN_US062_off_T
# DFNTT = np.concatenate((DFNTT,DFN_LA922_off),axis=0)

# SPMTT = NDC_4C
# SPMTT = np.concatenate((SPMTT,NDC_4C_off),axis=0)
# DFNTT = DFN_4C
# DFNTT = np.concatenate((DFNTT,DFN_4C_off),axis=0)

trainXTT = np.zeros((len(SPMTT),7))
trainXTT[:,0] = SPMTT[:,0] # 0: Vt
trainXTT[:,1] = SPMTT[:,1] # 1: Vb
trainXTT[:,2] = SPMTT[:,2] # 2: Vs
trainXTT[:,3] = SPMTT[:,3] # 3: V1
trainXTT[:,4] = SPMTT[:,4] # 4: SoC
trainXTT[:,5] = SPMTT[:,5] # 5: R0
trainXTT[:,6] = SPMTT[:,6] # 6: Current

# trainXTT = SPMTT
trainYTT = DFNTT

# deltaVTT = DFNTT.reshape(len(DFNTT),1) - SPMTT[:,0].reshape(len(SPMTT),1)
# trainYTT = deltaVTT

# normalizedXTT = scaler1.transform(trainXTT)
# normalizedY = scaler2.transform(trainY)
# trainXTT = normalizedXTT
# trainY = normalizedY

In [39]:
# # 2-tap delay
# init1 = SPMTT[0][6].reshape(1,1)
# init2 = SPMTT[0][7].reshape(1,1)
# testPredict_ = []
# for i in range(0,len(trainYTT)):
#     testX = SPMTT[i,:]
#     if i==0:
#         testX[6] = init1[0][0]
#         testX[7] = init2[0][0]
#     elif i==1:
#         testX[6] = init2[0][0]
#         testX[7] = testPredict_[i-1][0]
#     else:
#         testX[6] = testPredict_[i-2][0]
#         testX[7] = testPredict_[i-1][0]
#     normalizedXtest = scaler1.transform(testX.reshape(1,len(testX)))
#     testPredict = model.predict(normalizedXtest)
#     testPredict = scaler2.inverse_transform(testPredict)
#     testPredict_.append(testPredict)
# testPredict_ = np.array(testPredict_).reshape(len(trainYTT),1)
# testScore = math.sqrt(mean_squared_error(trainYTT, testPredict_))
# print("Test Score: RMSE " + str(testScore))

In [81]:
testPredict = SPMTT[0][6].reshape(1,1)
testPredict_ = []
for i in range(0,len(DFNTT)):
    testX = SPMTT[i,:]
    testX[6] = testPredict[0][0]
    normalizedXtest = scaler1.transform(testX.reshape(1,len(testX)))
    testPredict = model.predict(normalizedXtest)
    testPredict = scaler2.inverse_transform(testPredict)
    testPredict_.append(testPredict)
testPredict_ = np.array(testPredict_).reshape(len(trainYTT),1)
testScore = math.sqrt(mean_squared_error(trainYTT, testPredict_))
print("Test Score: RMSE " + str(testScore))

Test Score: RMSE 0.03190542375146388


In [ ]:
trainPredict = model.predict(normalizedXTT)
trainPredict = scaler2.inverse_transform(trainPredict.reshape(len(trainPredict),1))
trainScore = math.sqrt(mean_squared_error(DFNTT.reshape(len(trainPredict),1), trainPredict.reshape(len(trainPredict),1)))
print("Train Score: RMSE " + str(trainScore))
# InitialScore = math.sqrt(mean_squared_error(DFNTT, SPMTT[:,0]))
# print("Initial Score: RMSE " + str(InitialScore))

In [ ]:
# np.savetxt('Hybrid_Exp_UDDS_Test.csv', SPMTT[:,0].reshape(len(SPMTT),1) + trainPredict.reshape((len(trainPredict),1)), delimiter=',')
np.savetxt('Hybrid_Exp_UDDS_Test.csv', trainPredict.reshape((len(trainPredict),1)), delimiter=',') 

In [ ]:
np.savetxt('NDC_Exp_UDDS_Test.csv', SPMTT[:,0].reshape(len(SPMTT),1), delimiter=',') 